<a href="https://colab.research.google.com/github/mandashivakumar/WellSync-AI/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load dataset
file_path = "/content/gym_members_exercise_tracking.csv"
data = pd.read_csv(file_path)

# Data Preprocessing
# Encoding categorical variables
label_encoders = {}
categorical_features = ["Gender", "Experience_Level", "Workout_Type"]

for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature])
    label_encoders[feature] = le

# Handling missing values by dropping them
data = data.dropna()

# Selecting feature columns
features = [
    "Age", "Gender", "Weight_(kg)", "Height_(m)", "Max_BPM", "Avg_BPM",
    "Resting_BPM", "Session_Duration (hours)", "Calories_Burned",
    "Fat_Percentage", "Water_Intake (liters)", "Workout_Frequency", "BMI"
]

# Splitting data into training and testing sets
X = data[features].values
y = data[["Workout_Type", "Calories_Burned"]].values  # Target variables

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y[:, 0])

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the Multi-Task Neural Network
class MultiTaskMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MultiTaskMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.3)

        # Output layers for multi-task learning
        self.workout_output = nn.Linear(32, num_classes)  # Classification (Workout_Type)
        self.calories_output = nn.Linear(32, 1)  # Regression (Calories_Burned)

    # Corrected indentation for the forward method
    def forward(self, x):
        # Ensure the input has the correct shape [batch_size, num_features]
        if x.dim() == 1:
            x = x.unsqueeze(0)  # Add batch dimension if missing

        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))

        workout = self.workout_output(x)  # Classification output
        calories = self.calories_output(x)  # Regression output
        return workout, calories

# Get number of unique classes in the target variable
num_classes = len(np.unique(y_train[:, 0]))

# Initialize the model
model = MultiTaskMLP(input_size=len(features), num_classes=num_classes)

# Define the optimizer and loss functions
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
classification_loss_fn = nn.CrossEntropyLoss()  # For Workout_Type
regression_loss_fn = nn.MSELoss()  # For Calories_Burned

# Training Loop with Accuracy Tracking
epochs = 50
batch_size = 32
train_loader = torch.utils.data.DataLoader(list(zip(X_train_tensor, y_train_tensor)), batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        workout_pred, calories_pred = model(batch_X)

        # Loss calculation
        classification_loss = classification_loss_fn(workout_pred, batch_y[:, 0].long())  # Workout_Type
        regression_loss = regression_loss_fn(calories_pred.squeeze(), batch_y[:, 1].float())  # Calories_Burned
        loss = classification_loss + regression_loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(workout_pred, 1)
        correct += (predicted == batch_y[:, 0].long()).sum().item()
        total += batch_y.size(0)

    train_accuracy = 100 * correct / total
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

# Model Evaluation on Test Set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    workout_pred, calories_pred = test_outputs
    _, test_predictions = torch.max(workout_pred, 1)
    test_accuracy = 100 * (test_predictions == y_test_tensor[:, 0].long()).sum().item() / y_test_tensor.size(0)

print(f"\nFinal Model Accuracy on Test Data: {test_accuracy:.2f}%")

# User Input Prediction
def predict_fitness():
    user_input = []
    for feature in features:
        if feature in categorical_features:
            value = input(f"Enter {feature}: ")
            value = label_encoders[feature].transform([value])[0]
            value = float(value)
        else:
            value = float(input(f"Enter {feature}: "))
        user_input.append(value)

    # Store Weight, Height, and Age for later use
    weight = user_input[features.index("Weight_(kg)")]
    height = user_input[features.index("Height_(m)")]
    age = user_input[features.index("Age")]

    # Scale the input
    user_input_scaled = scaler.transform([user_input])
    user_input_tensor = torch.tensor(user_input_scaled, dtype=torch.float32)

    # Get predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        # Reshape the input to have a batch size of 1
        #user_input_tensor = user_input_tensor.unsqueeze(0)  # Remove this line
        workout_pred, calories_pred = model(user_input_tensor)

    # Decode the predictions
    workout_type = torch.argmax(workout_pred, dim=1).item()
    calories_burned = calories_pred.item()

    # Inverse transform for calories
    calories_burned = (calories_burned * scaler.scale_[features.index("Calories_Burned")]) + scaler.mean_[features.index("Calories_Burned")]

    # Output predictions
    print(f"\nPredicted Workout Type: {label_encoders['Workout_Type'].inverse_transform([workout_type])[0]}")
    print(f"Predicted Calories Burned: {calories_burned:.2f}")

    return workout_type, calories_burned, weight, height, age  # Return additional values

# Recommend Workout
def recommend_workout(workout_type):
    recommendations = {
        "Yoga": ["Sun Salutation", "Warrior Pose", "Tree Pose"],
        "HIIT": ["Burpees", "Mountain Climbers", "Jump Squats"],
        "Strength": ["Deadlifts", "Bench Press", "Squats"],
        "Cardio": ["Running", "Cycling", "Jump Rope"]
    }
    workout_name = label_encoders["Workout_Type"].inverse_transform([workout_type])[0]
    print(f"\nRecommended Exercises for {workout_name}:")
    for exercise in recommendations.get(workout_name, ["General Fitness Routine"]):
        print(f"- {exercise}")

# Provide Nutrition Guidance
def provide_nutrition_guidance(calories_burned, weight, height, age):
    bmr = 10 * weight + 6.25 * height * 100 - 5 * age + 5  # Basal Metabolic Rate
    daily_calories = bmr + calories_burned
    print(f"\nPersonalized Nutrition Guidance:")
    print(f"Your estimated daily calorie intake should be around {daily_calories:.2f} calories.")

# Main Function to Run All Features
def main():
    # Get predictions and user inputs
    workout_type, calories_burned, weight, height, age = predict_fitness()

    # Recommend workout
    recommend_workout(workout_type)

    # Provide nutrition guidance using stored values
    provide_nutrition_guidance(calories_burned, weight, height, age)

# Run the program
if __name__ == "__main__":
    main()

Epoch 5, Loss: 21064848.1875, Training Accuracy: 25.32%
Epoch 10, Loss: 15116458.0625, Training Accuracy: 26.22%
Epoch 15, Loss: 6835408.9531, Training Accuracy: 25.96%
Epoch 20, Loss: 2027847.3594, Training Accuracy: 23.14%
Epoch 25, Loss: 471579.5605, Training Accuracy: 26.22%
Epoch 30, Loss: 547755.7461, Training Accuracy: 24.42%
Epoch 35, Loss: 426402.2051, Training Accuracy: 26.09%
Epoch 40, Loss: 344716.4736, Training Accuracy: 25.96%
Epoch 45, Loss: 348848.4658, Training Accuracy: 24.16%
Epoch 50, Loss: 389138.4321, Training Accuracy: 26.48%

Final Model Accuracy on Test Data: 24.62%
Enter Age: 33
Enter Gender: Female
Enter Weight_(kg): 65
Enter Height_(m): 1.6
Enter Max_BPM: 178
Enter Avg_BPM: 165
Enter Resting_BPM: 62
Enter Session_Duration (hours): 1.5
Enter Calories_Burned: 876
Enter Fat_Percentage: 18
Enter Water_Intake (liters): 1.7
Enter Workout_Frequency: 3
Enter BMI: 20

Predicted Workout Type: Cardio
Predicted Calories Burned: 270933.79

Recommended Exercises for Cardi

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [22]:
torch.save(model.state_dict(), "model.pth")

In [26]:
model = MultiTaskMLP(input_size=len(features), num_classes=len(np.unique(data["Workout_Type"])))
model.load_state_dict(torch.load("model.pth"))
model.eval()

MultiTaskMLP(
  (fc1): Linear(in_features=13, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (workout_output): Linear(in_features=32, out_features=4, bias=True)
  (calories_output): Linear(in_features=32, out_features=1, bias=True)
)

In [ ]:
!streamlit run app.py --server.headless true




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.93.212:8501

